In [34]:
import librosa
import numpy as np
from io import BytesIO
import joblib
import speech_recognition as sr
from sklearn.preprocessing import MinMaxScaler

# Modeli yükleyin
model = joblib.load('../Model/VoiceModel.pkl')  # Modelin bulunduğu path'i ayarlayın

# Mikrofon için tanıyıcıyı başlatın
recognizer = sr.Recognizer()

# Mikrofonla ses kaydedin
with sr.Microphone() as source:
    print("Recording... Speak now.")
    recognizer.adjust_for_ambient_noise(source)  # Çevresel gürültüyü dikkate al
    audio = recognizer.listen(source)  # Ses kaydını al
    print("Recording finished, processing...")

# Ses verilerini baytlara çıkarın
audio_data = audio.get_wav_data()

# BytesIO kullanarak audio'yu librosa ile yükleyin
audio_io = BytesIO(audio_data)
audio, _ = librosa.load(audio_io, sr=16000)

# Özellikleri çıkartma (MFCC, Chroma, vb.)
mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)  # 40 MFCC özelliği
mfcc_mean = np.mean(mfcc, axis=1)  # MFCC'nin ortalamasını al
mfcc_std = np.std(mfcc, axis=1)    # MFCC'nin standart sapmasını al

# Chroma özellikleri
chroma = librosa.feature.chroma_stft(y=audio, sr=16000)
chroma_mean = np.mean(chroma, axis=1)

# RMS (Root Mean Square) enerji
rms = librosa.feature.rms(y=audio)
rms_mean = np.mean(rms)

# Özellikleri doğru sırayla birleştirelim (52 özellik)
features = np.concatenate((
    mfcc_mean[2:19],   # MFCC_3'ten MFCC_19'a kadar (index 2-18)
    mfcc_mean[21:25],  # MFCC_21'den MFCC_24'e kadar (index 21-24)
    mfcc_mean[26:28],  # MFCC_27 ve MFCC_28 (index 26-27)
    mfcc_mean[34:40],  # MFCC_35'ten MFCC_40'a kadar (index 34-39)
    mfcc_std[1:2],     # MFCC_Std_2 (index 1)
    mfcc_std[2:4],     # MFCC_Std_3 ve MFCC_Std_4 (index 2-3)
    mfcc_std[4:6],     # MFCC_Std_5 ve MFCC_Std_6 (index 4-5)
    mfcc_std[7:8],     # MFCC_Std_8 (index 7)
    mfcc_std[9:11],    # MFCC_Std_10 ve MFCC_Std_11 (index 9-10)
    mfcc_std[11:13],   # MFCC_Std_12 ve MFCC_Std_13 (index 11-12)
    mfcc_std[13:15],   # MFCC_Std_14 ve MFCC_Std_15 (index 13-14)
    chroma_mean[:5],   # Chroma_1'den Chroma_5'e kadar (index 0-4)
    chroma_mean[6:10], # Chroma_7'den Chroma_10'a kadar (index 6-9)
    [rms_mean]         # RMS özelliği
))

# Özellik vektörünün uzunluğunu kontrol edelim
print("Feature vector length:", len(features))  # Bu 52 olmalı

# Özellikleri normalize etme
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features.reshape(-1, 1)).flatten()

# Özelliklerin doğru formatta (1, 52) olduğundan emin olalım
features_normalized = features_normalized.reshape(1, -1)

# Modelle tahmin yapma
prediction = model.predict(features_normalized)
print(f"Predicted label: {prediction[0]}")


Recording... Speak now.
Recording finished, processing...
Feature vector length: 51


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 51 features, but KNeighborsClassifier is expecting 52 features as input.